## Машинное обучение, ВМК МГУ

## Практическое задание 2

### Общая информация
Дата выдачи: 9 октября 2019

Максимальная оценка: 10 баллов + 1 бонусный балл

Мягкий дедлайн: 23:59MSK 23 октября (за каждый день просрочки снимается 1 балл)

Жесткий дедлайн: 23:59MSK 30 октября.

### О задании

В этом задании вы:
- Познакомитесь с методом решения задачи регрессии на основе метода ближайших соседей.
- Реализуете алгоритм kNN для задачи регрессии.
- Изучите методы работы с категориальными и текстовыми переменными.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.


### Формат сдачи
Для сдачи задания переименуйте получившийся файл *.ipynb в соответствии со следующим форматом: homework-practice-02-Username.ipynb, где Username — ваша фамилия и имя на латинице именно в таком порядке (например, homework-practice-02-ivanov.ipynb).

Далее отправьте этот файл на anytask в соответсвующий раздел.

In [1]:
import numpy as np
import pandas as pd

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования New York City Airbnb Open Data: https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data#AB_NYC_2019.csv

В данной задаче предлагается предсказать цену на съем квартиры в зависимости от её параметров.

In [2]:
data = pd.read_csv('./data/AB_NYC_2019.csv')
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
data.shape

(48895, 16)

In [4]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

id 48895
name 47906
host_id 37457
host_name 11453
neighbourhood_group 5
neighbourhood 221
latitude 19048
longitude 14718
room_type 3
price 674
minimum_nights 109
number_of_reviews 394
last_review 1765
reviews_per_month 938
calculated_host_listings_count 47
availability_365 366


In [5]:
data.isna().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [6]:
data

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


Как видите, в данных есть пропуски. Не забудьте обработать их

Заполним пропуски в колонках **name** и **host_name** значением '-1', которое будет сигнализировать о пропуске в колонке.

In [7]:
data.loc[data['name'].isna(), 'name'] = '-1'
data.loc[data['host_name'].isna(), 'host_name'] = '-1'

С колонкой $last\_review$ поступим иначе:
* $last\_review$ - рассмотрим объект $x$ такой, что $x["last\_review"] = nan$ и $x["neighbourhood"] = neigh$. Тогда $x["last\_review"]$ заполняем наиболее встречаемым значением в колонке $last\_review$ среди всех объектов $x_{i}$ тренировочной выборки: $x_{i}["neighbourhood"] = neigh$

In [8]:
%%time
data['last_review'] = data['last_review'].fillna('-1')
# is_last_review_nan = data['last_review'].isna()
# for neigh in data[is_last_review_nan]['neighbourhood']:
#     is_neigh = data['neighbourhood'] == neigh
#     if data[is_neigh].shape[0] == 1:
#         data.loc[is_last_review_nan, 'last_review'] = '-1'
#     else:
#         data.loc[is_neigh & is_last_review_nan, 'last_review'] = data[is_neigh]['last_review'].value_counts().idxmax()

CPU times: user 13.8 ms, sys: 0 ns, total: 13.8 ms
Wall time: 12.4 ms


А для колонки **$reviews\_per\_month$** возьмем не наиболее встречаемое значение, а среднее:

In [9]:
%%time
data['reviews_per_month'] = data['reviews_per_month'].fillna(0)
# is_reviews_per_month_nan = data['reviews_per_month'].isna()
# for neigh in data[is_last_review_nan]['neighbourhood']:
#     is_neigh = data['neighbourhood'] == neigh
#     if data[is_neigh].shape[0] == 1:
#         data.loc[is_reviews_per_month_nan, 'reviews_per_month'] = -1
#     else:
#         data.loc[is_neigh & is_reviews_per_month_nan, 'reviews_per_month'] = data[is_neigh]['reviews_per_month'].mean()

CPU times: user 1.58 ms, sys: 194 µs, total: 1.77 ms
Wall time: 1.14 ms


Разобъем данные на обучение и контроль.

Для тестирования модели без категориальных признаков:

In [10]:
from sklearn.model_selection import train_test_split
X_train_non_cat, X_test_non_cat, y_train_non_cat, y_test_non_cat = train_test_split(data.drop(columns=['price',
                                                                                              'last_review',
                                                                                              'name',
                                                                                              'host_name',
                                                                                              'neighbourhood_group',
                                                                                              'neighbourhood',
                                                                                              'room_type']),
                                                                                    data[['price']],
                                                                                    test_size=0.3, random_state=241)

Для тестирования только категориальных признаков:

In [11]:
from sklearn.model_selection import train_test_split
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(data.drop(columns=['price',
                                                                                        'id',
                                                                                                       'host_id',
                                                                                                       'latitude',
                                                                                                       'longitude',
                                                                                                       'minimum_nights',
                                                                                                       'number_of_reviews',
                                                                                                       'reviews_per_month',
                                                                                                       'calculated_host_listings_count',
                                                                                                       'availability_365']),
                                                                                    data[['price']],
                                                                                    test_size=0.3, random_state=241)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['price']),
                                                    data[['price']],
                                                    test_size=0.3, random_state=241)

## Часть 1: Алгоритм kNN в задаче регрессии

<b> 1.1 (1.5 балла) </b>
Реализуйте класс `KNNRegressor`, который используя метод k ближайших соседей решает задачу регрессии. Для решение данной задачи, необходимо найти $N_k$ - k соседей, и после использовать значения их целевых переменных для предсказания:
\begin{align}
y = \frac{1}{k}\sum_{n \in N_k}w_n y_n,
\end{align}

где $w_n$ - вес каждого соседа. 

При этом `KNNRegressor` может работать в 2 режимах:
 - $uniform$ - ближайшие соседи учитываются с одинаковыми весами.
 - $distance$ - вес ближайших соседей зависит от расстояния
 
Сигнатуру методов при желании можно менять.

In [167]:
from typing import Union, Callable, Iterable, Optional
from sklearn.neighbors import NearestNeighbors


class KNNRegressor:
    def __init__(self, n_neighbors: int,
                 metric: Union[str, Callable],
                 mode: str = 'uniform',
                test_block_size = 10):
        """
        params:
            n_neighbors: number of neighbors
            metric: metric to use for distance computation
            mode: 'uniform' or 'distance'
            'uniform' - all points in each neighborhood are weighted equally
            'distance' - weight points by the inverse of their distance
            test_block_size: block size for finding neighbors and predict (for saving memory)
        """
        
        self.metric = metric
        self.__nn = None
        if self.metric not in ['overlap', 'flattened_overlap', 'log_overlap']:
            self.__nn = NearestNeighbors(n_neighbors=n_neighbors, metric=self.metric)
        self.__mode = mode
        self.n_neighbors = n_neighbors
        self.test_block_size = test_block_size
        self.y_train = None
        self.neigh_idxs = None
        self.distances = None

    def fit(self, X: np.array, y: np.array) -> None:
        """
            X: data
            y: labels
        """
        
        self.y_train = y
        if self.__nn is None:
            self.X_train = X
        else:
            self.__nn.fit(X, y)
        self.eps = 1e-5
        
    def flattened_overlap_df(self, x_df, z_df):
        result = np.zeros(x_df.shape[0], z_df.shape[0])
        for i, x_row in enumerate(x_df):
            result[i, :] = np.apply_along_axis(lambda z: flattened_overlap(x_row, z), axis=1)
        return result
    
        
    def find_kneighbors(self, X, return_distance):
        """
        params:
            * X - objects sample
            * return_distance - bool variable

        return values:
            * If return_distance == True:
                * tuple with two numpy array with size (X.shape[0], k), where:
                  [i, j] elem of first array must be the distance between
                  i-th object and his j-th nearest neighbour
                  [i, j] elem of second array must be the index of j-th nearest neighbour to i-th object
            * If return_distance == False:
                * only second array
        """


        if self.__nn is not None:
            self.distances, \
                self.neigh_idxs = self.__nn.kneighbors(X, n_neighbors=self.n_neighbors)
        else:
            if self.metric == 'overlap':
                self.distances = overlap(X, self.X_train)
            elif self.metric == 'flattened_overlap':
                self.distances = flattened_overlap_df(X, self.X_train, self.X_train)
            elif self.metric == 'log_overlap':
                self.distances = log_overlap_df(X, self.X_train, X, self.X_train)
            self.neigh_idxs = np.argsort(self.distances,
                                         axis=1)[:, :self.n_neighbors]
            if return_distance:
                self.distances = self.distances[np.arange(self.distances.shape[0])[:, None],
                                                self.neigh_idxs]
        if return_distance:
            return self.distances, self.neigh_idxs
        return self.neigh_idxs

    def predict(self, X: np.array, n_neighbors: Optional[int] = None) -> np.array:
        """
            X: data
            n_neighbors: number of neighbors
        """
        if n_neighbors is not None:
            n_neighbors = np.minimum(self.n_neighbors, n_neighbors)
        else:
            n_neighbors = self.n_neighbors
        if self.test_block_size > X.shape[0]:
            self.test_block_size = X.shape[0]
        preds = np.zeros(X.shape[0])
        split_size = X.shape[0] // self.test_block_size + \
                     int(X.shape[0] % self.test_block_size != 0)
        curr_idx = 0
        for i, split in enumerate(np.array_split(X, split_size)):
            self.distances, self.neigh_idxs = self.find_kneighbors(split,
                                                                   return_distance=True
                                                                  )
            for j, idx in enumerate(self.neigh_idxs):
                if self.__mode == 'distance':
                    weights = 1 / (self.distances[j] + self.eps)
                    pred = np.sum(self.y_train.values[idx].reshape(-1) * weights) / np.sum(weights)
                else:
                    pred = np.sum(self.y_train.values[idx].reshape(-1)) / n_neighbors
                preds[j + curr_idx] = pred
            curr_idx += split.shape[0]
        return preds
    
    
    def predict_without_testblock(self, X):
        """
        params:
            * X - test objects

        return values:

        """
        
        self.distances, self.neigh_idxs = self.find_kneighbors(X,
                                                               return_distance=True
                                                              )
        preds = np.zeros(X.shape[0])
        for j, idx in enumerate(self.neigh_idxs):
            if self.__mode == 'distance':
                weights = 1 / (self.distances[j] + self.eps)
                pred = np.sum(self.y_train[idx] * weights) / np.sum(weights)
            else:
                pred = np.sum(self.y_train[idx]) / self.n_neighbors
            preds[j] = pred
        return preds
    
    
    def predict_opt(self, X):
        """
        params:
            * X - test objects

        return values:

        """

        preds = np.zeros(X.shape[0])
        print(self.n_neighbors)
        for j, idx in enumerate(self.neigh_idxs[:, :self.n_neighbors]):
            if self.__mode == 'distance':
                weights = 1 / (self.distances[j, :self.n_neighbors] + self.eps)
                counts = np.bincount(self.y_train[idx],  weights)
            else:
                counts = np.bincount(self.y_train[idx])
            preds[j] = np.argmax(counts)
        return preds

In [14]:
def rmse(y_true, y_valid):
    return (np.sum((y_valid - y_true) ** 2) / len(y_true)) ** (1/2)

In [15]:
k_neighbors = 10
knn = KNNRegressor(k_neighbors, metric='euclidean', mode='euclidean', test_block_size=500)

In [16]:
%%time
knn.fit(X_train_non_cat, y_train_non_cat)
preds = knn.predict(X_test_non_cat, 10)

CPU times: user 385 ms, sys: 3.68 ms, total: 388 ms
Wall time: 387 ms


In [17]:
rmse(y_test_non_cat.values.reshape(-1), preds)

224.97411992865153

## Часть 2: Категориальные признаки

<b>2.1 (1 балл)</b>
Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на [третьем семинаре](https://github.com/mmp-mmro-team/mmp_mmro_fall_2019/blob/master/lecture-notes/Sem03_knn.pdf). Не забудьте, что KNNRegressor должен уметь работать с этими функциями расстояния. Как вариант, можно реализовать метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html).

In [18]:
# import pyximport; pyximport.install() Попытки переписать метрики на Cython оказались безуспешными
# import overlap_cython

In [19]:
def overlap(x, z):
    """
    x, z - pandas dataframes with shape lxd
    """
    x = x.to_numpy(copy=True)
    z = z.to_numpy(copy=True)
    is_equal = x[:, None] != z
    return np.sum(is_equal, axis=2)

def flattened_overlap(x, z, df):
    f_x = np.zeros(x.shape[0])
    f_z = np.zeros(z.shape[0])
    p2_sum_arr = np.zeros(x.shape[0])
    p2_sum_arr_z = np.zeros(z.shape[0])
    for i, x_val in enumerate(x):
        f_x = np.sum(df.iloc[:, i] == x_val)
    for i, z_val in enumerate(z):
        f_z[i] = np.sum(df.iloc[:, i] == z_val)
    p_x = f_x / df.shape[0]

    for i, x_val in enumerate(x):
        f_x = df.iloc[:, i].value_counts()
        p2 = f_x
        p2 = p2 * (p2 - 1) / (df.shape[0] * (df.shape[0] - 1))
#         print(f_x[x_val])
        p2_sum_arr[i] = np.sum(p2 * (f_x.values <= f_x[x_val]))
    result = np.sum(x != z) + np.sum((x == z) * p2_sum_arr)
    return result


# def flattened_overlap(x, z, df):
#     f_x = np.zeros(x.shape[0])
#     p2_sum_arr = np.zeros(x.shape[0])
#     df = df.to_numpy()
#     f_x = np.sum(df == x[None, :], axis=0)
#     p_x = f_x / df.shape[0]
#     p2_sum_arr = np.sum(p_x * (f_x - 1) / (df.shape[0] - 1) * (p_x <= p_x[:, None]), axis=1)
#     result = np.sum(x != z) + np.sum((x == z) * p2_sum_arr)
#     return result


def log_overlap(x, z, df_x, df_z):
    result = 0
    f_x = np.sum(df_x == x[None, :], axis=0)
    f_z = np.sum(df_z == z[None, :], axis=0)
    result = np.sum((x != z) * np.log(f_x + 1) * np.log(f_z + 1))
    return result

In [22]:
flattened_overlap(data.loc[22796, ['neighbourhood_group', 'neighbourhood', 'room_type']],
            data.loc[26190, ['neighbourhood_group', 'neighbourhood', 'room_type']],
            data.loc[:, ['neighbourhood_group', 'neighbourhood', 'room_type']])
            #data.loc[:, ['host_name', 'neigbhourhood_group', 'neighbourhood', 'room_type', 'last_review']])

3.0

In [28]:
log_overlap(X_train_cat.iloc[1, 2:5],# 'last_review']],
            X_train_cat.iloc[2, 2:5],# 'last_review']],
            data.loc[:, ['neighbourhood_group', 'neighbourhood', 'room_type']],
            data.loc[:, ['neighbourhood_group', 'neighbourhood', 'room_type']])

222.20320784530432

In [22]:
X_train_cat.iloc[0, 1:5]

host_name                      Sara
neighbourhood_group        Brooklyn
neighbourhood            Kensington
room_type              Private room
Name: 39319, dtype: object

In [23]:
X_train_cat.iloc[1, 1:5]

host_name                      Chelsea
neighbourhood_group          Manhattan
neighbourhood                 Gramercy
room_type              Entire home/apt
Name: 22796, dtype: object

In [24]:
X_train_cat.iloc[2, 1:5]

host_name                     Lidia
neighbourhood_group          Queens
neighbourhood             Sunnyside
room_type              Private room
Name: 26190, dtype: object

In [29]:
k_neighbors = 10
knn = KNNRegressor(k_neighbors, metric='overlap', mode='distance', test_block_size=500)

In [30]:
%%time
knn.fit(X_train_cat.iloc[:, :], y_train_cat.iloc[:, :])
preds = knn.predict(X_test_cat.iloc[:, :], 10)

CPU times: user 44.1 s, sys: 1.96 s, total: 46.1 s
Wall time: 46.1 s


In [31]:
rmse(y_test_non_cat.values.reshape(-1), preds)

217.01271722720145

In [32]:
#228.2563985959649 # flattened 
#227.45479221155136 # log_overlap
#222.72230579804977 # overlap

In [42]:
# def flattened_overlap_df(x_df, z_df, df):
#         result = np.zeros((x_df.shape[0], z_df.shape[0]))
#         for i, x_row in enumerate(x_df.iterrows()):
#             x_row = np.array(x_row[1])
#             result[i, :] = np.apply_along_axis(lambda z: flattened_overlap(x_row, z, df), axis=1, arr=z_df)
#         return result
    
def flattened_overlap_df(x_df, z_df, df):
        result = np.zeros((x_df.shape[0], z_df.shape[0]))
        for i, x_row in enumerate(x_df.iterrows()):
            print(i)
            x_row = np.array(x_row[1])
            result[i, :] = np.apply_along_axis(lambda z: flattened_overlap(x_row, z, df), axis=1, arr=z_df)
            print(i)
        return result
    
def log_overlap_df(x_df, z_df, full_df_x, full_df_z):
        result = np.zeros((x_df.shape[0], z_df.shape[0]))
        for i, x_row in enumerate(x_df.iterrows()):
            x_row = np.array(x_row[1])
            print(i)
            result[i, :] = np.apply_along_axis(lambda z: log_overlap(x_row, z,
                                                                     full_df_x, full_df_z),
                                               axis=1,
                                               arr=z_df)
#             if (i % 1) == 0:
            print(i)
        return result

In [43]:
%%time
log_overlap_distances = log_overlap_df(X_test_cat, X_train_cat, X_test_cat, X_train_cat)

0


KeyboardInterrupt: 

In [49]:
%%time
flattened_overlap_distances = flattened_overlap_df(X_test_cat, X_train_cat, pd.concat([X_train_cat, X_test_cat]))

0


KeyboardInterrupt: 

In [338]:
X_train.ix[423, 'host_name']

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


'Jonathan'

In [27]:
#241.02891795616793

## Воспользуемся LabelEncoder для того, чтобы преобразовать категориальные признаки

In [50]:
from sklearn.preprocessing import LabelEncoder

In [51]:
le = LabelEncoder()
for column in X_train_cat.columns:
    all_classes_in_col = np.hstack([X_train_cat[column].unique(), X_test_cat[column].unique()])
    le.fit(all_classes_in_col)
    X_train_cat.loc[:, column] = le.transform(X_train_cat.loc[:, column])
    X_test_cat.loc[:, column] = le.transform(X_test_cat.loc[:, column])


In [52]:
knn = KNNRegressor(10, metric='overlap', mode='uniform', test_block_size=1000)
test_size = X_train_cat.shape[0]
knn.fit(X_train_cat, y_train_cat)

In [53]:
%%time
preds = knn.predict(X_test_cat.iloc[:test_size])

CPU times: user 9.62 s, sys: 1.73 s, total: 11.4 s
Wall time: 11.4 s


In [54]:
rmse(y_test_cat.iloc[:test_size].values.reshape(-1), preds)
#195.55411220426942
#218.08062593048203
#

218.08062593048203

In [55]:
knn = KNNRegressor(10, metric='flattened_overlap', mode='uniform', test_block_size=10)
knn.fit(X_train_cat, y_train_cat)

In [27]:
%%time
preds = knn.predict(X_test_cat)

TypeError: flattened_overlap() missing 1 required positional argument: 'df'

In [ ]:
rmse(y_test_cat.values.reshape(-1), preds)

In [ ]:
knn = KNNRegressor(10, metric=log_overlaperlap, mode='uniform', test_block_size=10)
knn.fit(X_train_cat, y_train_cat)

In [ ]:
%%time
preds = knn.predict(X_test_cat)

In [56]:
rmse(y_test_cat.values.reshape(-1), preds)

218.08062593048203

<b>2.2 (1 балл)</b> Найдите все категориальные признаки в данных. Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Качество измеряйте с помощью RMSE.

Какая функция расстояния оказалась лучшей? Почему?

In [31]:
# Ваш код здесь

<b>2.3 (1 балл) бонус</b> Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какого удалось достичь уровня качества?

In [32]:
# Ваш код здесь

<b>2.4 (2.5 балла)</b> Отойдем ненадолго от задачи регрессии и перейдём к задаче классификации: будем определять, являеться ли квартира дорогой $(target = 1)$ или дешевой $(target = 0)$. Будем считать дорогими квариры, цена которых выше среднего.

In [57]:
data['target'] = (data.price > data.price.mean()).astype(int)

Реализуйте счетчики, которые заменят категориальные признаки на вещественные.

А именно, для каждого категориального признака $f_j(x)$ необходимо сделать следующее:
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
\begin{align}
counts_j(c) = \sum_{i=1}^l [f_j(x_i) = c]
\end{align}
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
\begin{align}
successes_j(c) = \sum_{i=1}^l[f_j(x_i) = c][y_i = +1].
\end{align}
3. Сглаженное отношение двух предыдущих величин:
\begin{align}
p_j(c) = \frac{successes_j(c) + a}{counts_j(c) + b},
\end{align}

где $a$ и $b$ - априорные счетчики (например, a = 1, b = 2).

In [58]:
def counts(x):
    """
    x - one categorical column with type - pandas series
    """
    
    x = x.to_numpy()
    return np.sum(x[:, None] == x, axis=1)


def successes(x, y):
    """
    x - one categorical column with type - pandas series
    y - target feature from df
    """
    
    x = x.to_numpy()
    y = y.to_numpy()
    result = np.sum((x[:, None] == x) * (y == 1), axis=1)
    return result
    

def flattened_c_s(x, y, a=1, b=2):
    """
    x - one categorical column with type - pandas series
    y - target feature from dataframe
    a - priori counter
    b - priori counter
    """
    
    return (successes(x, y) + a) / (counts(x) + b)

In [59]:
%%time
cnts = counts(X_train_cat.iloc[:, 1])

CPU times: user 831 ms, sys: 166 ms, total: 996 ms
Wall time: 996 ms


In [60]:
cnts[:50]

array([ 65,  25,   7, 108,   1,   1,  79,   9,   3,   3,   4,  20, 230,
        27,   4,  28,  37,  21,  14,  53,  10,  89,   2, 104,  80,   6,
        37,   1,   2,  45, 114,  14,   1,   1,  27,  70,  18,   1,  30,
        22,   2,   1,   3,   6,   6, 103,  49,  41,   9, 278])

In [61]:
%%time
scss = successes(data['room_type'][:10], data['target'][:10])

CPU times: user 578 µs, sys: 0 ns, total: 578 µs
Wall time: 583 µs


In [62]:
scss

array([0, 2, 0, 2, 2, 2, 0, 0, 0, 2])

In [63]:
%%time
flattened_c_s(data['neighbourhood_group'][:10], data['target'][:10])

CPU times: user 688 µs, sys: 0 ns, total: 688 µs
Wall time: 663 µs


array([0.2       , 0.33333333, 0.33333333, 0.2       , 0.33333333,
       0.33333333, 0.2       , 0.33333333, 0.33333333, 0.33333333])

In [322]:
def counters(df_cat, y):
    df_copy = df_cat.copy()
    for column in df_copy.columns:
        print(column)
        df_copy[column + '_counts'] = counts(df_copy[column])
        df_copy[column + '_successes'] = successes(df_copy[column], y)
        df_copy[column + '_flattened_c_s'] = flattened_c_s(df_copy[column], y)
        df_copy.drop(column, axis=1, inplace=True)
    return df_copy

In [323]:
from sklearn.model_selection import train_test_split
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(data.drop(columns=['price',
                                                                                       'target',
                                                                                        'id',
                                                                                        'host_id',
                                                                                        'latitude',
                                                                                        'longitude',
                                                                                        'minimum_nights',
                                                                                        'number_of_reviews',
                                                                                        'reviews_per_month',
                                                                                        'calculated_host_listings_count',
                                                                                        'availability_365']),
                                                                                    data[['target']],
                                                                                    test_size=0.3, random_state=241)

In [324]:
le = LabelEncoder()
for column in X_train_cat.columns:
    all_classes_in_col = np.hstack([X_train_cat[column].unique(), X_test_cat[column].unique()])
    le.fit(all_classes_in_col)
    X_train_cat.loc[:, column] = le.transform(X_train_cat.loc[:, column])
    X_test_cat.loc[:, column] = le.transform(X_test_cat.loc[:, column])

In [325]:
%%time
counters(X_train_cat, y_train)

name
host_name
neighbourhood_group
neighbourhood
room_type
last_review
CPU times: user 30.4 s, sys: 10 s, total: 40.4 s
Wall time: 36.3 s


,name_counts,name_successes,name_flattened_c_s,host_name_counts,host_name_successes,host_name_flattened_c_s,neighbourhood_group_counts,neighbourhood_group_successes,neighbourhood_group_flattened_c_s,neighbourhood_counts,neighbourhood_successes,neighbourhood_flattened_c_s,room_type_counts,room_type_successes,room_type_flattened_c_s,last_review_counts,last_review_successes,last_review_flattened_c_s
39319,1,0,0.333333,65,0,0.014925,14061,0,0.000071,125,0,0.007874,15563,0,0.000064,510,0,0.001953
22796,1,0,0.333333,25,0,0.037037,15179,0,0.000066,242,0,0.004098,17846,0,0.000056,9,0,0.090909
26190,1,0,0.333333,7,0,0.111111,3969,0,0.000252,271,0,0.003663,15563,0,0.000064,936,0,0.001066
8161,1,0,0.333333,108,0,0.009091,15179,0,0.000066,1224,0,0.000816,17846,0,0.000056,6993,0,0.000143
32473,1,0,0.333333,1,0,0.333333,15179,0,0.000066,1281,0,0.000779,17846,0,0.000056,396,0,0.002513
35951,1,0,0.333333,1,0,0.333333,15179,0,0.000066,1387,0,0.000720,15563,0,0.000064,26,0,0.035714
25954,1,0,0.333333,79,0,0.012346,3969,0,0.000252,178,0,0.005556,17846,0,0.000056,948,0,0.001053
18513,1,0,0.333333,9,0,0.090909,14061,0,0.000071,2636,0,0.000379,17846,0,0.000056,6993,0,0.000143
47680,1,0,0.333333,3,0,0.200000,15179,0,0.000066,774,0,0.001289,17846,0,0.000056,6993,0,0.000143
30397,1,0,0.333333,3,0,0.200000,14061,0,0.000071,2748,0,0.000364,15563,0,0.000064,16,0,0.055556


Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Достаточно взять $n = 3$.

## Реализуем kfold:

In [354]:
def kfold(indices, n_folds=3):
    """
    params:
        * n - objects amount in sample
        * n_folds - folds amount
    return values:
        * list with size n_folds, where every element is tuple of two 1D numpy array:
            * first array contains indices of train samples
            * second array contains indices of validation samples
    """

#     np.random.shuffle(indices)
    n = len(indices)
    size_of_one_fold = int(n / n_folds) + (n < n_folds)
    size_with_folds = size_of_one_fold * n_folds
    out_elements_amount = len(indices) - size_with_folds
    train_test_idx_list = []
    for i in range(n_folds):
        test_idx = indices[i * size_of_one_fold:(i + 1) * size_of_one_fold]
        if out_elements_amount > 0:
            test_idx = np.append(test_idx, n - out_elements_amount)
            out_elements_amount -= 1
        train_idx = np.array(list((set(indices) - set(test_idx))))
        train_test_idx_list.append((train_idx, test_idx))
    return train_test_idx_list

In [355]:
def counts_with_folds(x, n_folds=3):
    """
    x - one categorical column with type - pandas series
    y - target feature from df
    n_folds - amount of folds
    """
    
    result = x.copy()
    result.iloc[:] = 0
    for help_indx, main_indx in kfold(x.index, n_folds):
        unique_elems = np.unique(x[help_indx])
        successes_per_fold = {x : 0 for x in unique_elems}
        for unique_elem in unique_elems:
            successes_per_fold[unique_elem] = np.sum((x[help_indx] == unique_elem))
            result[np.array((x[main_indx] == unique_elem)[x[main_indx] == unique_elem].index)] = successes_per_fold[unique_elem]
    return result

In [307]:
%%time
counts_with_folds(X_train_cat.loc[:50, 'neighbourhood'])

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/pandas/core/series.py:942: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


CPU times: user 923 ms, sys: 0 ns, total: 923 ms
Wall time: 922 ms


39319     17
22796     50
26190     53
8161     216
32473    231
35951    245
25954     25
18513    478
47680    132
30397    485
17974    243
11554    245
43397    104
20388    243
34422    132
12891     65
16475    485
14004     47
19160    216
24003    144
20439     62
22695    243
28335    216
48220    339
33428     62
14246    485
45829     56
14021     33
39027    478
39903    485
        ... 
28676    143
4554      57
7581     135
62       213
9495      49
20052    494
16392    261
33568     60
14416    157
7924       2
12303    165
39575    261
12489     67
30302    135
35853     55
11010     26
42895     56
24320     49
16305    107
46264     67
38678     32
8499     494
10569    213
47635     24
12278    143
13151     26
10517    325
23861    483
175       57
50         0
Name: neighbourhood, Length: 9337, dtype: int64

In [317]:
def successes_with_folds(x, y, n_folds=3):
    """
    x - one categorical column with type - pandas series
    y - target feature from df
    n_folds - amount of folds
    """
    
    result = x.copy()
    result.iloc[:] = 0
    for help_indx, main_indx in kfold(x.index, n_folds):
        unique_elems = np.unique(x[help_indx])
        successes_per_fold = {x : 0 for x in unique_elems}
        for unique_elem in unique_elems:
            successes_per_fold[unique_elem] = np.sum((x[help_indx] == unique_elem) * (y[help_indx] == 1))
            result[np.array((x[main_indx] == unique_elem)[x[main_indx] == unique_elem].index).reshape(-1)] = successes_per_fold[unique_elem]
    return result

In [309]:
successes_with_folds(X_train_cat.loc[:, 'neighbourhood_group'], y_train_cat['target'])

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/pandas/core/computation/expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


39319    2002
22796    4587
26190     316
8161     4587
32473    4587
35951    4587
25954     316
18513    2002
47680    4587
30397    2002
17974    4587
11554    4587
43397    4587
20388    4587
34422    4587
12891    2002
16475    2002
14004    4587
19160    4587
24003    2002
20439    2002
22695    4587
28335    4587
48220    4587
33428    2002
14246    2002
45829    4587
14021     316
39027    2002
39903    2002
         ... 
40438    1968
48111      32
30338    4596
3476       32
19639    1968
34582    1968
12256    4596
14074    1968
14512    1968
34404     312
6103     1968
9798     1968
18672     312
36266    1968
8960     1968
48803    4596
2543     1968
48667    4596
15316     312
4407     1968
3379     4596
25015     312
37407    1968
21785    4596
4987     4596
41432    4596
8690     4596
144      1968
10987       0
12716       0
Name: neighbourhood_group, Length: 34226, dtype: int64

In [382]:
def flattened_c_s_with_folds(x, y, n_folds=3):
    """
    x - one categorical column with type - pandas series
    y - target feature from df
    n_folds - amount of folds
    """
    
    result = x.copy()
    result.iloc[:] = 0
    for help_indx, main_indx in kfold(x.index, n_folds):
        unique_elems = np.unique(x[help_indx])
        successes_per_fold = {x : 0 for x in unique_elems}
        for unique_elem in unique_elems:
            successes_per_fold[unique_elem] = np.sum((x[help_indx] == unique_elem) * (y.loc[help_indx] == 1))
            result[np.array((x[main_indx] == unique_elem)[x[main_indx] == unique_elem].index)] = successes_per_fold[unique_elem]
    return result

In [381]:
flattened_c_s_with_folds(X_train_cat.loc[:, 'neighbourhood_group'], y_train_cat['target'])

IndexError: index 34226 is out of bounds for axis 0 with size 34226

39319    0
22796    1
26190    0
8161     1
32473    1
35951    0
25954    0
18513    0
47680    1
30397    0
17974    0
11554    1
43397    1
20388    0
34422    0
12891    0
16475    0
14004    0
19160    1
24003    1
20439    0
22695    0
28335    1
48220    0
33428    0
14246    0
45829    1
14021    0
39027    0
39903    1
        ..
40438    0
48111    0
30338    1
3476     0
19639    0
34582    0
12256    1
14074    0
14512    0
34404    0
6103     0
9798     0
18672    0
36266    0
8960     0
48803    0
2543     0
48667    1
15316    1
4407     0
3379     1
25015    0
37407    0
21785    1
4987     0
41432    1
8690     0
144      0
10987    0
12716    0
Name: target, Length: 34226, dtype: int64

In [312]:
def counters_with_folds(df_cat, y):
    df_copy = df_cat.copy()
    for column in df_copy.columns:
        df_copy[column + '_counts'] = counts_with_folds(df_copy.loc[:, column])
        df_copy[column + '_successes'] = successes_with_folds(df_copy.loc[:, column], y)
        df_copy[column + '_flattened_c_s'] = flattened_c_s_with_folds(df_copy.loc[:, column], y)
        df_copy.drop(column, axis=1, inplace=True)
    return df_copy

In [313]:
counters_with_folds(X_train_cat.iloc[:50, :], y_train_cat['target'][:50])

,name_counts,name_successes,name_flattened_c_s,host_name_counts,host_name_successes,host_name_flattened_c_s,neighbourhood_group_counts,neighbourhood_group_successes,neighbourhood_group_flattened_c_s,neighbourhood_counts,neighbourhood_successes,neighbourhood_flattened_c_s,room_type_counts,room_type_successes,room_type_flattened_c_s,last_review_counts,last_review_successes,last_review_flattened_c_s
39319,0,0,0,0,0,0,13,4,4,0,0,0,11,0,0,0,0,0
22796,0,0,0,0,0,0,17,7,7,0,0,0,20,11,11,1,0,0
26190,0,0,0,0,0,0,3,0,0,0,0,0,11,0,0,2,1,1
8161,0,0,0,0,0,0,17,7,7,2,1,1,20,11,11,7,3,3
32473,0,0,0,0,0,0,17,7,7,0,0,0,20,11,11,0,0,0
35951,0,0,0,0,0,0,17,7,7,1,1,1,11,0,0,0,0,0
25954,0,0,0,0,0,0,3,0,0,0,0,0,20,11,11,3,0,0
18513,0,0,0,0,0,0,13,4,4,3,0,0,20,11,11,7,3,3
47680,0,0,0,0,0,0,17,7,7,1,0,0,20,11,11,7,3,3
30397,0,0,0,0,0,0,13,4,4,4,2,2,11,0,0,0,0,0


Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

## Without folding

In [327]:
from sklearn.metrics import roc_auc_score

In [328]:
X_train_cat_without_folding = counters(X_train_cat, y_train_cat['target'])

name
host_name
neighbourhood_group
neighbourhood
room_type
last_review


In [329]:
X_train_cat_without_folding

,name_counts,name_successes,name_flattened_c_s,host_name_counts,host_name_successes,host_name_flattened_c_s,neighbourhood_group_counts,neighbourhood_group_successes,neighbourhood_group_flattened_c_s,neighbourhood_counts,neighbourhood_successes,neighbourhood_flattened_c_s,room_type_counts,room_type_successes,room_type_flattened_c_s,last_review_counts,last_review_successes,last_review_flattened_c_s
39319,1,0,0.333333,65,19,0.298507,14061,2990,0.212686,125,13,0.110236,15563,1003,0.064504,510,132,0.259766
22796,1,1,0.666667,25,9,0.370370,15179,6891,0.453989,242,131,0.540984,17846,9388,0.526053,9,3,0.363636
26190,1,0,0.333333,7,1,0.222222,3969,465,0.117351,271,17,0.065934,15563,1003,0.064504,936,310,0.331557
8161,1,1,0.666667,108,42,0.390909,15179,6891,0.453989,1224,527,0.430669,17846,9388,0.526053,6993,2647,0.378556
32473,1,1,0.666667,1,1,0.666667,15179,6891,0.453989,1281,583,0.455183,17846,9388,0.526053,396,122,0.309045
35951,1,0,0.333333,1,0,0.333333,15179,6891,0.453989,1387,626,0.451404,15563,1003,0.064504,26,10,0.392857
25954,1,0,0.333333,79,23,0.296296,3969,465,0.117351,178,22,0.127778,17846,9388,0.526053,948,266,0.281053
18513,1,0,0.333333,9,0,0.090909,14061,2990,0.212686,2636,382,0.145186,17846,9388,0.526053,6993,2647,0.378556
47680,1,1,0.666667,3,2,0.600000,15179,6891,0.453989,774,164,0.212629,17846,9388,0.526053,6993,2647,0.378556
30397,1,0,0.333333,3,1,0.400000,14061,2990,0.212686,2748,873,0.317818,15563,1003,0.064504,16,1,0.111111


In [404]:
knn_regressor = KNNRegressor(10, metric='euclidean')
weights_list = ['uniform', 'distance']
k_max = 15
k_list = np.arange(k_max)
best_score_param = {'score': None, 'param': None}
for weights in weights_list:
    knn_regressor = KNNRegressor(k_max, metric='euclidean', mode=weights, test_block_size=10000)
    knn_regressor.fit(X_train_cat_without_folding, y_train_cat['target'].values)
    knn_regressor.find_kneighbors(X_train_cat_without_folding, return_distance=True)
    for k in k_list[1:]:
        knn_regressor.n_neighbors = k
        pred = knn_regressor.predict_opt(X_train_cat_without_folding)
        roc_auc = roc_auc_score(pred, y_train_cat['target'].values)
        print(roc_auc)
        if best_score_param['score'] is None:
            best_score_param['score'] = roc_auc
            best_score_param['param'] = str(weights) + '_' + str(k)            
        elif abs(best_score_param['score'] - 0.5) < abs(roc_auc - 0.5):
            best_score_param['score'] = roc_auc
            best_score_param['param'] = str(weights) + '_' + str(k)
        
        
        
    

1
0.9997874083502546
2
0.9384596013818409
3
0.8863840616789482
4
0.8790431978651786
5
0.8534446456855811
6
0.8541861526247078
7
0.8360738586322749
8
0.8396109917926218
9
0.8242535708313307
10
0.8290700731708955
11
0.8201951219787937
12
0.8217085359566624
13
0.8135231948062135
14
0.8135092012738581
1
0.9997874083502546
2
0.9997874083502546
3
0.9997874083502546
4
0.9997874083502546
5
0.9997605822639342
6
0.9998680178697779
7
0.9998680178697779
8
0.9998680178697779
9
0.999894931495335
10
0.999894931495335
11
0.999894931495335
12
0.999894931495335
13
0.999894931495335
14
0.999894931495335


In [405]:
best_score_param

{'score': 0.999894931495335, 'param': 'distance_9'}

<b>2.5 (1 балл)</b> Вернемся к задаче регрессии. Утверждается, что для задачи регрессии можно также сделать преобразование категориальных признаков в действительные числа. Для этого достаточно для каждого значения признака $f_j$ вычислить:
\begin{align}
p_j(c) = g(T_i | f_j(x_i) = c),
\end{align}

где $T_i$ - значения целевой переменной объекта $x_i$. Функция $g$ - среднее (mean) или среднеквадратичное отклонение (std).

Закодируйте категориальные признаки обоими способами и найдите значение RMSE. Используйте евклидову метрику для поиска ближайших соседей. Для какой функции $g$ значение RMSE лучше? Почему?

In [45]:
def calculate_p_j(f_j, g_func=np.mean):
    print(f_j[:, None] == f_j)
    return

## Часть 3: Текстовые признаки

<b>3.1 (2 балла)</b> Перейдем от категориальным признаков к текстовым. Рассмотрим 2 способа преобразования текста в действительные числа:
- Мешок слов (Bag of Words)
- TF-IDF

[Здесь](https://scikit-learn.org/stable/modules/feature_extraction.html) вы можете прочитать про их применение в Питоне.

Сравните оба способа на задаче регресси. Какую лучше метрику использовать: евклидову или косинусную меру? Постройте графики зависимости качества решения задачи от способа преобразования, метрики и количества соседей. Мера качества - RMSE.

Объясните полученные результаты.

Перед преобразованием не забудьте уменьшить размер словаря. Например, это можно сделать за счет приведения всех слов к одному регистру и удаления [стопслов](https://en.wikipedia.org/wiki/Stop_words) (артиклей, предлогов, союзов).

## One register

In [62]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_set = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/paniquex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Delete punctuation and made one register

In [222]:
data_text = data.copy()
data_text['name'] = data_text['name'] + data_text['host_name'] + data_text['neighbourhood'] + data_text['neighbourhood_group'] + \
data_text['room_type'] + data_text['last_review']
data_text['name'] = data_text['name'].str.lower()
data_text['name'] = data_text['name'].str.split()
data_text['name'] = data_text['name'].apply(lambda x: [elem for elem in x if elem not in stopwords_set])

In [223]:
data_text['name'] = data_text['name'].apply(lambda x: [elem for elem in x if elem not in list(string.punctuation)])

In [224]:
data_text['name'] = data_text['name'].str.join(' ')

In [225]:
from sklearn.feature_extraction.text import CountVectorizer

In [226]:
vectorizer = CountVectorizer()

In [227]:
data_with_text_features_bag = vectorizer.fit_transform(data_text['name'])

In [231]:
X_train_with_text, X_test_with_text, y_train_with_text, y_test_with_text = train_test_split(data_with_text_features_bag,
                                                                                    data[['price']].values,
                                                                                    test_size=0.3, random_state=241)

In [229]:
knn_regressor = KNNRegressor(10, metric='euclidean', mode='distance', test_block_size=100000)
knn_regressor.fit(X_train_with_text, y_train_with_text[:, 0])

In [230]:
rmse(knn_regressor.predict_without_testblock(X_test_with_text), y_test_with_text[:, 0])

209.7913373357753

In [232]:
y_test_with_text

array([[129],
       [316],
       [165],
       ...,
       [200],
       [ 85],
       [100]])

In [233]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [234]:
vectorizer_tfid = TfidfVectorizer()
data_with_text_features_tfidf = vectorizer_tfid.fit_transform(data_text['name'])

In [235]:
X_train_with_text, X_test_with_text, y_train_with_text, y_test_with_text = train_test_split(data_with_text_features_tfidf,
                                                                                    data[['price']].values,
                                                                                    test_size=0.3, random_state=241)

In [240]:
knn_regressor = KNNRegressor(10, metric='euclidean', mode='distance', test_block_size=100000)
knn_regressor.fit(X_train_with_text, y_train_with_text[:, 0])

In [241]:
rmse(knn_regressor.predict_without_testblock(X_test_with_text), y_test_with_text[:, 0])

202.82080212782293

In [ ]:
#202.7376205627374

<b>3.2 (1 балл)</b> Используя все доступные признаки, решите задачу регрессии. Для категориальных и текстовых признаков выберите лучшие преобразования. Повлияло ли добавление количественного признака на метрику качества?

In [ ]:
# Ваш код здесь

## Часть 4: Выводы


In [ ]:
# Ваши выводы здесь (ノ°∀°)ノ⌒･*:.｡. .｡.:*･゜ﾟ･*☆